In [1]:
import pandas as pd

In [20]:
from utils.lipht_visualization import word_frequency_barplot, topic_distribution_barplot
from utils.lipht_lda_utils import PrepareDictionary, df_lda_features, get_topics_and_probability, get_lda_topics, df_lda_preprocessing, TrainLDAModel
from utils.lipht_data import getEngine

In [3]:
engine =  getEngine('LIPHT-VM-01','Akademikernes_MSCRM_Addition')

In [4]:
query="""
SELECT *
  FROM [Akademikernes_MSCRM_Addition].[out].[LDA_Messages_persisted]
  """
df_scope = pd.read_sql(query, engine)

In [5]:
df_scope.head()

,ThreadID,ThreadStartDate,ThreadEndDate,ThreadSubject,FirstMessageID,FirstMessageBody,FirstMemberMessageID,FirstMemberMessageBody,FirstResponseFromAKAToMemberMessageID,FirstResponseFromAKAToMemberMessageBody,...,LastMessageID,LastMessageBody,ThreadInitiatedBy,ThreadClass,HasMemberMessage,HasResponseFromAKAToMember,HasResponseFromMemberToAKA,ThreadResponsibleDepartment,ThreadResponsibleDepartmentTeam,ThreadResponsibleDepartmentTeamGroup
0,645A5871-F7B5-E811-82B4-0050569118B0,2018-09-11 19:18:26,2018-09-17 14:17:11,Difference i timerne i juli måned,6A5A5871-F7B5-E811-82B4-0050569118B0,Kære Thomas Hauerslev Andersen\nVi skal hver m...,DBE20A48-69BA-E811-864D-005056AD2D14,"Hej Dorte,\n \nJeg ved ærlig talt ikke lige hv...",03E41A5A-84BA-E811-864D-005056AD2D14,Hej Thomas\nJeg henlægger sagen vedrørende de ...,...,03E41A5A-84BA-E811-864D-005056AD2D14,Hej Thomas\nJeg henlægger sagen vedrørende de ...,AKA,"AKA initiates thread, the member responds and ...",1,1,1,Forsikring,Kvalitetsteam,Kvalitetsteam
1,E7748CFB-27B6-E811-82B4-0050569118B0,2018-09-12 01:05:48,2018-09-13 09:12:52,Dit cv på jobnet.dk mangler oplysninger for at...,EE748CFB-27B6-E811-82B4-0050569118B0,Kære Mohamad Khaled Abdel Rahman\n \nVi skrive...,93D623B0-70B6-E811-864D-005056AD2D14,Kære Akademikernes A-kasse.\n \nJeg kan se at ...,89E6572B-35B7-E811-864D-005056AD2D14,"Kære Mohamad Khaled Abdel Rahman,\nTak for din...",...,89E6572B-35B7-E811-864D-005056AD2D14,"Kære Mohamad Khaled Abdel Rahman,\nTak for din...",AKA,"AKA initiates thread, the member responds and ...",1,1,1,Jobmatch,Team Rådighed,Team Rådighed
2,7A18B9F5-48B6-E811-82B4-0050569118B0,2018-09-12 05:01:58,2018-09-13 12:03:58,Giv oplysninger om jobsøgningen og bliv ved me...,8218B9F5-48B6-E811-82B4-0050569118B0,"Kære Lena Arndal \n \nVi ved, at du er i fuld ...",70BD297A-88B6-E811-864D-005056AD2D14,"Hej \nJeg kan se, at oplysninger fra min joblo...",126F1F17-4DB7-E811-864D-005056AD2D14,"Kære Lena Arndal,\nTak for din besked.\nMøder,...",...,126F1F17-4DB7-E811-864D-005056AD2D14,"Kære Lena Arndal,\nTak for din besked.\nMøder,...",AKA,"AKA initiates thread, the member responds and ...",1,1,1,Jobmatch,Team Rådighed,Team Rådighed
3,E1BAC568-59B6-E811-82B4-0050569118B0,2018-09-12 06:59:45,2018-09-12 07:20:05,Hvornår blev du færdiguddannet?,E7BAC568-59B6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nVi har modt...,A460B937-5BB6-E811-864D-005056AD2D14,"Okay. Jeg tror, jeg har været lidt forvirret.\...",6874B044-5CB6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nTak for dit...,...,6874B044-5CB6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nTak for dit...,AKA,"AKA initiates thread, the member responds and ...",1,1,1,Forsikring,Udbetalingsteam 1 - CPR 01 til 10,Udbetalingsteam
4,778B1D72-5BB6-E811-82B4-0050569118B0,2018-09-12 07:14:11,2018-09-25 06:36:08,Din timeindberetning i maj og juni måned,7D8B1D72-5BB6-E811-82B4-0050569118B0,Kære Cecilie Bech Fejfer\nTak for dit svar ved...,C3A97562-ECBF-E811-864D-005056AD2D14,"Hej Gitte,\n \nbeklager sent svar. Jeg glemte ...",F3E66444-8DC0-E811-82B4-0050569118B0,Kære Cecilie Bech Fejfer\nJeg skal desværre ha...,...,F3E66444-8DC0-E811-82B4-0050569118B0,Kære Cecilie Bech Fejfer\nJeg skal desværre ha...,AKA,"AKA initiates thread, the member responds and ...",1,1,1,Forsikring,Udbetalingsteam 1 - CPR 01 til 10,Udbetalingsteam


In [14]:
# df_scope = pd.read_pickle('data/LDA_Messages_persisted_with_language.pkl')
# df_scope.to_pickle('LDA_Messages_persisted.pkl')
df_scope = pd.read_pickle('LDA_Messages_persisted.pkl')

In [7]:
departmentteam = 'Udbetalingsteam'
initiatedby = 'AKA'
n_gram = 3
sample_size= 10000
no_above = 0.01
no_below= 2 # filter out tokens that appear in less than 15 documents
random_state=1
lda_num_topics = 50
lda_chunksize = 1000
lda_passes = 50

log = {
    'departmentteam': departmentteam,
    'initiatedby': initiatedby,
    'n_gram': n_gram,
    'no_above': no_above,
    'no_below': no_below,
    'random_state': random_state,
    'lda_num_topics': lda_num_topics,
    'lda_chunksize': lda_chunksize,
    'lda_passes': lda_passes
}

In [8]:
df_team = df_scope[(df_scope['FirstMessageBody'].isnull()==False) & (df_scope['ThreadResponsibleDepartmentTeamGroup'].str.contains(departmentteam)==True)].copy(deep=True)
log['rows'] = df_team.shape[0]

In [9]:
df_lda_preprocessing(df_team, 'FirstResponseFromAKAToMemberMessageBody')

In [23]:
df_team.head(2)

,ThreadID,ThreadStartDate,ThreadEndDate,ThreadSubject,FirstMessageID,FirstMessageBody,FirstMemberMessageID,FirstMemberMessageBody,FirstResponseFromAKAToMemberMessageID,FirstResponseFromAKAToMemberMessageBody,...,text_Questionmarks,text_1Question,text_Exclamationmarks,tokenized_text,stemmed_text,bigrams,trigrams,ngrams,stopwords_removed,bow
3,E1BAC568-59B6-E811-82B4-0050569118B0,2018-09-12 06:59:45,2018-09-12 07:20:05,Hvornår blev du færdiguddannet?,E7BAC568-59B6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nVi har modt...,A460B937-5BB6-E811-864D-005056AD2D14,"Okay. Jeg tror, jeg har været lidt forvirret.\...",6874B044-5CB6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nTak for dit...,...,0,False,0,"[peter, brandthenriksen, tak, for, dit, hurtig...","[pet, brandthenriks, tak, for, dit, hurt, svar...","[ansøgning om, behandle din, brandthenriksen t...","[ansøgning om dagpenge, behandle din ansøgning...","[ansøgning om, behandle din, brandthenriksen t...","[ansøgning om, behandle din, brandthenriksen t...","[(11, 1)]"
4,778B1D72-5BB6-E811-82B4-0050569118B0,2018-09-12 07:14:11,2018-09-25 06:36:08,Din timeindberetning i maj og juni måned,7D8B1D72-5BB6-E811-82B4-0050569118B0,Kære Cecilie Bech Fejfer\nTak for dit svar ved...,C3A97562-ECBF-E811-864D-005056AD2D14,"Hej Gitte,\n \nbeklager sent svar. Jeg glemte ...",F3E66444-8DC0-E811-82B4-0050569118B0,Kære Cecilie Bech Fejfer\nJeg skal desværre ha...,...,1,True,0,"[bech, fejfer, skal, desværre, have, oplysning...","[bech, fejf, skal, desvær, hav, oplysning, om,...","[at undersøge, bech fejfer, desværre have, fej...","[bech fejfer skal, desværre have oplysninger, ...","[at undersøge, bech fejfer, desværre have, fej...","[at undersøge, bech fejfer, desværre have, fej...","[(19, 1), (27, 1), (45, 1), (47, 1), (49, 1)]"


In [24]:
df_team['text'][3]

'peter brandt-henriksen\ntak for dit hurtige svar.\n vil behandle din ansøgning om dagpenge hurtigst muligt.\nhav en god dag.'

In [25]:
df_team['ngrams'][3]

['ansøgning om',
 'behandle din',
 'brandthenriksen tak',
 'dagpenge hurtigst',
 'din ansøgning',
 'dit hurtige',
 'en god',
 'for dit',
 'god dag',
 'hav en',
 'hurtige svar',
 'hurtigst muligt',
 'muligt hav',
 'om dagpenge',
 'peter brandthenriksen',
 'svar vil',
 'tak for',
 'vil behandle',
 'ansøgning om dagpenge',
 'behandle din ansøgning',
 'brandthenriksen tak for',
 'dagpenge hurtigst muligt',
 'din ansøgning om',
 'dit hurtige svar',
 'en god dag',
 'for dit hurtige',
 'hav en god',
 'hurtige svar vil',
 'hurtigst muligt hav',
 'muligt hav en',
 'om dagpenge hurtigst',
 'peter brandthenriksen tak',
 'svar vil behandle',
 'tak for dit',
 'vil behandle din',
 'pet',
 'brandthenriks',
 'tak',
 'for',
 'dit',
 'hurt',
 'svar',
 'vil',
 'behandl',
 'din',
 'ansøgning',
 'om',
 'dagpeng',
 'hurt',
 'mul',
 'hav',
 'en',
 'god',
 'dag']

In [26]:
dictionary, corpus = PrepareDictionary(df_team, 'ngrams', no_above, no_below, log)

Found 1363352 words.
Left with 100000 words.
bow column added to df


In [21]:
%%time
LDAmodel_scope = TrainLDAModel(corpus, dictionary, lda_num_topics, 4, lda_chunksize, lda_passes, random_state, log)

Wall time: 19min 45s


In [22]:
log

{'departmentteam': 'Udbetalingsteam',
 'initiatedby': 'AKA',
 'n_gram': 3,
 'no_above': 0.01,
 'no_below': 2,
 'random_state': 1,
 'lda_num_topics': 50,
 'lda_chunksize': 1000,
 'lda_passes': 50,
 'rows': 63626,
 'words_preprocess': 1363352,
 'words_postprocess': 100000,
 'train_start': datetime.datetime(2018, 10, 15, 12, 0, 18, 344858),
 'train_end': datetime.datetime(2018, 10, 15, 12, 20, 3, 729736)}

In [27]:
topic_distribution_barplot(LDAmodel_scope, df_team)

In [28]:
df_team.head(1)

,ThreadID,ThreadStartDate,ThreadEndDate,ThreadSubject,FirstMessageID,FirstMessageBody,FirstMemberMessageID,FirstMemberMessageBody,FirstResponseFromAKAToMemberMessageID,FirstResponseFromAKAToMemberMessageBody,...,text_1Question,text_Exclamationmarks,tokenized_text,stemmed_text,bigrams,trigrams,ngrams,stopwords_removed,bow,lda_features
3,E1BAC568-59B6-E811-82B4-0050569118B0,2018-09-12 06:59:45,2018-09-12 07:20:05,Hvornår blev du færdiguddannet?,E7BAC568-59B6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nVi har modt...,A460B937-5BB6-E811-864D-005056AD2D14,"Okay. Jeg tror, jeg har været lidt forvirret.\...",6874B044-5CB6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nTak for dit...,...,False,0,"[peter, brandthenriksen, tak, for, dit, hurtig...","[pet, brandthenriks, tak, for, dit, hurt, svar...","[ansøgning om, behandle din, brandthenriksen t...","[ansøgning om dagpenge, behandle din ansøgning...","[ansøgning om, behandle din, brandthenriksen t...","[ansøgning om, behandle din, brandthenriksen t...","[(11, 1)]","[0.032860055565834045, 0.028788715600967407, 0..."


In [29]:
topics = get_topics_and_probability(df_team, LDAmodel_scope, lda_num_topics, 40)

In [30]:
topics

,Probability,TopWords,Topic
0,0.053286,"0.035*""annul"" + 0.030*""lund"" + 0.027*""tilbagem...",0.0
1,0.043815,"0.057*""sket"" + 0.045*""prøv"" + 0.044*""ændring"" ...",1.0
2,0.045448,"0.049*""hvorfor"" + 0.037*""mandag"" + 0.036*""mads...",2.0
3,0.049799,"0.020*""drift"" + 0.019*""angiv"" + 0.019*""forvent...",3.0
4,0.043021,"0.041*""mari"" + 0.038*""redegør"" + 0.028*""maria""...",4.0
5,0.042939,"0.038*""nyuddanned"" + 0.037*""stud"" + 0.034*""kla...",5.0
6,0.020789,"0.072*""aftalt"" + 0.056*""forklaring"" + 0.026*""h...",6.0
7,0.025060,"0.060*""indtil"" + 0.037*""nødt"" + 0.031*""dansk"" ...",7.0
8,0.020996,"0.040*""then"" + 0.026*""first"" + 0.022*""year"" + ...",8.0
9,0.030288,"0.062*""hverdag"" + 0.048*""truk"" + 0.044*""nemkon...",9.0


In [ ]:
# topics.to_sql(name='topics_{}_{}'.format(dataset,initiatedby),con=engine , schema='input', if_exists='replace', index=False)

In [31]:
get_lda_topics(df_team, LDAmodel_scope, lda_num_topics,20)

,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,...,Topic # 40,Topic # 41,Topic # 42,Topic # 43,Topic # 44,Topic # 45,Topic # 46,Topic # 47,Topic # 48,Topic # 49
0,annul,sket,hvorfor,drift,mari,nyuddanned,aftalt,indtil,then,hverdag,...,overfør,filtyp,vejledningsbrev,befordring,holiday,aktivitet,gdag,kollektiv,pågæld,tilbagebetaling
1,lund,prøv,mandag,angiv,redegør,stud,forklaring,nødt,first,truk,...,hun,enkelt,seniorjob,kurs,stop,lønarbejd,vacation,ferielukning,ægtefæl,privat
2,tilbagemelding,ændring,mads,forvent,maria,klar,henlæg,dansk,year,nemkonto,...,termin,pdf,okay,jobplan,benyt,påfør,january,ferieperiod,book,opload
3,louis,møl,undersøg,forskel,ansættelsesforhold,indberetning,differenc,flyt,than,rest,...,nå,email,mark,sktilekort,honorar,kontrollabelt,overskyd,idag,indflyd,løntilskudsjob
4,gået,par,kristens,kl,sagsbehandling,lagt,dertil,adres,memb,disposition,...,post,indehold,pålag,joblog,thes,kraft,claus,jakobs,forløb,oprind
5,angå,adgang,fravær,cvr,underskriv,status,fin,kraft,they,afvikl,...,frederiks,vedhæftning,timestid,opdat,websit,trådt,brand,tro,betrag,typ
6,afmelding,ok,slut,spørg,beg,tilmelding,færdiggjort,ifølg,what,christian,...,anledning,tilhtm,ansættelsesbrev,lyd,earned,ukontrollabelt,bang,påbegynd,administr,herund
7,østergaard,holm,ols,set,umiddelbart,vigt,mellemlig,jacobs,contract,betalt,...,stod,n,hus,kursus,genoptag,bemærk,who,dagpengekort,lønindtæg,d
8,jfr,plad,mellem,lønseddel,indgår,skift,kontanthjælp,ene,sinc,kjær,...,imagegif,zip,sagd,meddel,sandgaard,tilføj,brink,omgang,glemt,regnskabsår
9,bort,fald,deltidsarbejd,kund,varsel,dagpengemodtag,payed,måt,rul,al,...,ole,jpg,confirmation,lind,vikar,minut,nguyen,arbejdsplad,tilbud,egent


In [ ]:
import pandas as pd
df_log = pd.DataFrame(log, index=['0'])
df_log

In [32]:
log

{'departmentteam': 'Udbetalingsteam',
 'initiatedby': 'AKA',
 'n_gram': 3,
 'no_above': 0.01,
 'no_below': 2,
 'random_state': 1,
 'lda_num_topics': 50,
 'lda_chunksize': 1000,
 'lda_passes': 50,
 'rows': 63626,
 'words_preprocess': 1363352,
 'words_postprocess': 100000,
 'train_start': datetime.datetime(2018, 10, 15, 12, 0, 18, 344858),
 'train_end': datetime.datetime(2018, 10, 15, 12, 20, 3, 729736)}